In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
import tensorflow as tf
from tensorflow.keras import layers, Model, datasets
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime

# Configuration
latent_dim = 128
batch_size = 64
epochs = 600
img_shape = (32, 32, 3)
save_interval = 3  # Save images every 3 epochs

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Create output directories
os.makedirs('gan_images', exist_ok=True)
os.makedirs('saved_models', exist_ok=True)

# Load and preprocess CIFAR-10 (cats only)
(x_train, y_train), (_, _) = datasets.cifar10.load_data()
x_train = x_train[y_train.flatten() == 3]  # Class 3 = cats
x_train = (x_train.astype('float32') - 127.5) / 127.5  # Normalize to [-1, 1]

# Data pipeline
dataset = tf.data.Dataset.from_tensor_slices(x_train)
dataset = dataset.shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [24]:
# Generator Model
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8*8*256, use_bias=False, input_dim=latent_dim),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Reshape((8, 8, 256)),

        layers.Conv2DTranspose(128, 4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(64, 4, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.ReLU(),

        layers.Conv2DTranspose(3, 4, strides=1, padding='same', activation='tanh')
    ])
    return model

# Discriminator Model
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, 4, strides=2, padding='same', input_shape=img_shape),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Conv2D(128, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Conv2D(256, 4, strides=2, padding='same'),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),

        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model


In [25]:
# Build models
generator = build_generator()
discriminator = build_discriminator()

# Optimizers
g_optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.5)
d_optimizer = tf.keras.optimizers.Adam(0.0002, beta_1=0.5)

# Loss function
cross_entropy = tf.keras.losses.BinaryCrossentropy()

def d_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

def g_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [26]:
# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as g_tape, tf.GradientTape() as d_tape:
        generated = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated, training=True)

        d_loss_value = d_loss(real_output, fake_output)
        g_loss_value = g_loss(fake_output)

    d_grads = d_tape.gradient(d_loss_value, discriminator.trainable_variables)
    g_grads = g_tape.gradient(g_loss_value, generator.trainable_variables)

    d_optimizer.apply_gradients(zip(d_grads, discriminator.trainable_variables))
    g_optimizer.apply_gradients(zip(g_grads, generator.trainable_variables))

    return d_loss_value, g_loss_value


In [27]:
# Image generation utility
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    predictions = (predictions * 127.5 + 127.5).numpy().astype('uint8')

    plt.figure(figsize=(10, 10))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    plt.savefig(f'gan_images/epoch_{epoch:04d}.png')
    plt.close()

# Training loop
fixed_seed = tf.random.normal([16, latent_dim])  # For consistent samples


In [28]:
for epoch in range(epochs):
    start = datetime.now()

    # Training
    for batch in dataset:
        d_loss_value, g_loss_value = train_step(batch)

    # Save models
    if (epoch + 1) % 10 == 0:
        generator.save(f'saved_models/generator_epoch_{epoch+1}.h5')
        discriminator.save(f'saved_models/discriminator_epoch_{epoch+1}.h5')

    # Generate images
    if (epoch + 1) % save_interval == 0:
        generate_and_save_images(generator, epoch + 1, fixed_seed)

    # Print progress
    print(f'Epoch {epoch+1}/{epochs} [D loss: {d_loss_value:.4f}] [G loss: {g_loss_value:.4f}] Time: {datetime.now()-start}')

# Final save
generator.save('saved_models/generator_final.h5')
discriminator.save('saved_models/discriminator_final.h5')

# Generate sample grid after training
generate_and_save_images(generator, epochs, fixed_seed)

Epoch 1/600 [D loss: 0.0415] [G loss: 6.8801] Time: 0:00:07.047766
Epoch 2/600 [D loss: 0.8243] [G loss: 3.1368] Time: 0:00:02.371804
Epoch 3/600 [D loss: 0.8552] [G loss: 1.3015] Time: 0:00:03.112746
Epoch 4/600 [D loss: 0.6825] [G loss: 1.8303] Time: 0:00:02.201003
Epoch 5/600 [D loss: 0.7702] [G loss: 1.3162] Time: 0:00:02.186020
Epoch 6/600 [D loss: 1.2885] [G loss: 0.9790] Time: 0:00:02.380813
Epoch 7/600 [D loss: 0.8792] [G loss: 1.0768] Time: 0:00:02.192363
Epoch 8/600 [D loss: 0.9792] [G loss: 1.2639] Time: 0:00:02.202914
Epoch 9/600 [D loss: 1.8270] [G loss: 0.7646] Time: 0:00:02.403732


Epoch 10/600 [D loss: 1.1939] [G loss: 0.7557] Time: 0:00:02.260032
Epoch 11/600 [D loss: 1.3806] [G loss: 0.8650] Time: 0:00:02.212500
Epoch 12/600 [D loss: 1.4450] [G loss: 0.8180] Time: 0:00:02.417729
Epoch 13/600 [D loss: 1.3667] [G loss: 0.8723] Time: 0:00:02.220453
Epoch 14/600 [D loss: 1.3526] [G loss: 0.8127] Time: 0:00:02.222897
Epoch 15/600 [D loss: 1.5291] [G loss: 0.6502] Time: 0:00:02.420592
Epoch 16/600 [D loss: 1.3093] [G loss: 0.6344] Time: 0:00:02.235005
Epoch 17/600 [D loss: 1.4093] [G loss: 0.6849] Time: 0:00:02.241172
Epoch 18/600 [D loss: 1.5421] [G loss: 0.5985] Time: 0:00:02.446218
Epoch 19/600 [D loss: 1.5571] [G loss: 0.6626] Time: 0:00:02.246096


Epoch 20/600 [D loss: 1.3402] [G loss: 0.6996] Time: 0:00:02.314662
Epoch 21/600 [D loss: 1.3906] [G loss: 0.7259] Time: 0:00:02.459265
Epoch 22/600 [D loss: 1.2374] [G loss: 0.7943] Time: 0:00:02.271830
Epoch 23/600 [D loss: 1.4291] [G loss: 0.8218] Time: 0:00:02.277589
Epoch 24/600 [D loss: 1.2096] [G loss: 0.7772] Time: 0:00:02.595107
Epoch 25/600 [D loss: 1.3357] [G loss: 0.6648] Time: 0:00:02.288935
Epoch 26/600 [D loss: 1.2496] [G loss: 0.7116] Time: 0:00:02.297354
Epoch 27/600 [D loss: 1.3730] [G loss: 0.6029] Time: 0:00:02.508660
Epoch 28/600 [D loss: 1.1083] [G loss: 0.7801] Time: 0:00:02.302876
Epoch 29/600 [D loss: 1.2586] [G loss: 0.6439] Time: 0:00:02.308482


Epoch 30/600 [D loss: 1.2861] [G loss: 0.8214] Time: 0:00:02.691867
Epoch 31/600 [D loss: 1.3481] [G loss: 0.9065] Time: 0:00:02.313668
Epoch 32/600 [D loss: 1.2623] [G loss: 0.7561] Time: 0:00:02.320692
Epoch 33/600 [D loss: 1.3795] [G loss: 0.6758] Time: 0:00:02.520942
Epoch 34/600 [D loss: 1.2511] [G loss: 0.7265] Time: 0:00:02.329520
Epoch 35/600 [D loss: 1.1293] [G loss: 1.0438] Time: 0:00:02.327508
Epoch 36/600 [D loss: 1.2868] [G loss: 0.6578] Time: 0:00:02.540685
Epoch 37/600 [D loss: 1.2642] [G loss: 1.0491] Time: 0:00:02.339589
Epoch 38/600 [D loss: 0.8816] [G loss: 1.3049] Time: 0:00:02.342435
Epoch 39/600 [D loss: 1.3320] [G loss: 0.6494] Time: 0:00:02.542342


Epoch 40/600 [D loss: 1.1335] [G loss: 1.1012] Time: 0:00:02.426891
Epoch 41/600 [D loss: 1.2326] [G loss: 0.7051] Time: 0:00:02.342686
Epoch 42/600 [D loss: 1.6331] [G loss: 0.8722] Time: 0:00:02.532831
Epoch 43/600 [D loss: 1.0994] [G loss: 0.8176] Time: 0:00:02.344659
Epoch 44/600 [D loss: 1.3600] [G loss: 0.7292] Time: 0:00:02.341930
Epoch 45/600 [D loss: 1.2820] [G loss: 0.7220] Time: 0:00:02.534015
Epoch 46/600 [D loss: 1.1392] [G loss: 0.8879] Time: 0:00:02.335933
Epoch 47/600 [D loss: 1.5172] [G loss: 0.8337] Time: 0:00:02.341775
Epoch 48/600 [D loss: 1.2058] [G loss: 0.8173] Time: 0:00:02.532734
Epoch 49/600 [D loss: 1.0609] [G loss: 0.8973] Time: 0:00:02.341710


...

Epoch 560/600 [D loss: 1.0301] [G loss: 1.2472] Time: 0:00:02.382360
Epoch 561/600 [D loss: 1.0610] [G loss: 1.0511] Time: 0:00:02.523990
Epoch 562/600 [D loss: 1.1493] [G loss: 1.0263] Time: 0:00:02.322679
Epoch 563/600 [D loss: 1.1350] [G loss: 1.0184] Time: 0:00:02.325928
Epoch 564/600 [D loss: 1.3790] [G loss: 1.0922] Time: 0:00:02.623969
Epoch 565/600 [D loss: 0.9788] [G loss: 1.2118] Time: 0:00:02.326587
Epoch 566/600 [D loss: 0.9954] [G loss: 1.1240] Time: 0:00:02.319165
Epoch 567/600 [D loss: 1.0263] [G loss: 1.3472] Time: 0:00:02.500415
Epoch 568/600 [D loss: 1.0110] [G loss: 1.0059] Time: 0:00:02.317567
Epoch 569/600 [D loss: 0.9757] [G loss: 1.0899] Time: 0:00:02.320490


Epoch 570/600 [D loss: 1.2044] [G loss: 1.1141] Time: 0:00:02.572459
Epoch 571/600 [D loss: 1.3877] [G loss: 1.3184] Time: 0:00:02.317387
Epoch 572/600 [D loss: 1.0623] [G loss: 1.2586] Time: 0:00:02.323973
Epoch 573/600 [D loss: 0.9854] [G loss: 1.2344] Time: 0:00:02.515123
Epoch 574/600 [D loss: 1.1418] [G loss: 1.3188] Time: 0:00:02.321572
Epoch 575/600 [D loss: 1.0767] [G loss: 1.2868] Time: 0:00:02.327027
Epoch 576/600 [D loss: 1.1557] [G loss: 1.2584] Time: 0:00:02.515323
Epoch 577/600 [D loss: 1.2935] [G loss: 1.2686] Time: 0:00:02.323347
Epoch 578/600 [D loss: 1.2546] [G loss: 1.2956] Time: 0:00:02.329382
Epoch 579/600 [D loss: 1.0399] [G loss: 1.2947] Time: 0:00:02.521595


Epoch 580/600 [D loss: 0.9899] [G loss: 1.3726] Time: 0:00:02.405908
Epoch 581/600 [D loss: 1.1986] [G loss: 1.2417] Time: 0:00:02.347384
Epoch 582/600 [D loss: 1.1135] [G loss: 1.1838] Time: 0:00:02.515598
Epoch 583/600 [D loss: 0.9279] [G loss: 1.3074] Time: 0:00:02.331599
Epoch 584/600 [D loss: 1.1788] [G loss: 0.9804] Time: 0:00:02.337984
Epoch 585/600 [D loss: 1.1388] [G loss: 1.3849] Time: 0:00:02.606519
Epoch 586/600 [D loss: 0.8513] [G loss: 1.2439] Time: 0:00:02.335530
Epoch 587/600 [D loss: 0.9117] [G loss: 1.3757] Time: 0:00:02.334685
Epoch 588/600 [D loss: 1.2604] [G loss: 1.1404] Time: 0:00:02.521578
Epoch 589/600 [D loss: 0.8534] [G loss: 1.2814] Time: 0:00:02.330893


Epoch 590/600 [D loss: 1.1571] [G loss: 1.0303] Time: 0:00:02.418241
Epoch 591/600 [D loss: 1.2536] [G loss: 1.2361] Time: 0:00:02.645253
Epoch 592/600 [D loss: 1.2504] [G loss: 1.3072] Time: 0:00:02.337127
Epoch 593/600 [D loss: 0.8484] [G loss: 1.2387] Time: 0:00:02.329335
Epoch 594/600 [D loss: 1.0570] [G loss: 1.2393] Time: 0:00:02.528435
Epoch 595/600 [D loss: 0.9316] [G loss: 1.3260] Time: 0:00:02.321716
Epoch 596/600 [D loss: 1.1656] [G loss: 1.1609] Time: 0:00:02.330982
Epoch 597/600 [D loss: 0.9991] [G loss: 1.2770] Time: 0:00:02.518880
Epoch 598/600 [D loss: 0.9565] [G loss: 1.2296] Time: 0:00:02.331902
Epoch 599/600 [D loss: 1.0061] [G loss: 1.2798] Time: 0:00:02.330793


Epoch 600/600 [D loss: 1.1259] [G loss: 1.2137] Time: 0:00:02.603937
